In [1]:
!git clone https://github.com/openai/human-eval.git
!pip install -e human-eval
import sys
sys.path.append('/kaggle/working/human-eval')

Cloning into 'human-eval'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 29 (delta 8), reused 4 (delta 4), pack-reused 8 (from 1)
Receiving objects: 100% (29/29), 54.20 KiB | 2.01 MiB/s, done.
Resolving deltas: 100% (9/9), done.
Obtaining file:///kaggle/working/human-eval
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114248 sha256=3f0fd88728d4ff325188ec15226d2b8b497bbdac2557d3668d29173a42ba59eb
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire
  Running setup.py develop for human-eval


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from human_eval.data import write_jsonl, read_problems
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [4]:
i=0

In [5]:
def split_and_trim_code(text: str):
    # Find the index where "def " starts
    def_index = text.find('def ')
    if def_index == -1:
        return ""  # return an empty string if "def " is not found

    # Extract the code starting from "def "
    code_after_def = text[def_index:]

    # Find the end of the "def " line
    def_line_end_index = code_after_def.find('\n')
    if def_line_end_index != -1:
        # Skip the entire line where "def " is found
        code_after_def = code_after_def[def_line_end_index+1:]
    
    # Find the index of "```" that comes after "def "
    end_code_index = code_after_def.find('```')
    
    # If "```" is found, return the code until just before "```"
    if end_code_index != -1:
        code = code_after_def[:end_code_index]
    else:
        # If no "```" is found, return the code until the end of the string
        code = code_after_def
    
    # Removing leading and trailing whitespaces and newlines
    return code.strip()

In [6]:
def generate_one_completion(prompt):
    global i 
    i += 1
    print("Generating")
    print(i)

    # Wrapping the prompt with instruction format tokens
    p = f"WRITE THE FULL COMPLETE FUNCTION (EG WITH def ....) END CODE WITH '```'. NOTE YOU ABSOLUTELY MUST END THE CODE WITH END CODE WITH '```' OR ELSE THE CODE WILL NOT BE INTERPRETTED!!!! {prompt}"

    # Generate a response using the model
    messages = [
        {"role": "system", "content": "You are a friendly chatbot"},
        {"role": "user", "content": p}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=3500
        )
    
    # Decode the generated ids to text
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # Print the text split and trimmed
    print("###" * 50)
    print(split_and_trim_code(response))
    return split_and_trim_code(response)

In [10]:
problems = read_problems()

# Define the number of samples to generate for each problem
num_samples_per_task = 10

# Generate samples
samples = [
    dict(task_id=task_id, completion=generate_one_completion(problem["prompt"]))
    for task_id, problem in problems.items()
    for _ in range(num_samples_per_task)
]

# Save generated samples in jsonl format
write_jsonl("/kaggle/working/qwen_samples_10.jsonl", samples)

Generating
165
######################################################################################################################################################
for i in range(len(numbers)):
        for j in range(i + 1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False
Generating
166
######################################################################################################################################################
for_close = set()
    for_far = []
    
    for_close.update(numbers)
    
    for_far.append(sorted(to_close))
    
    return len(set(range(len(for_far)))) < 2
Generating
167
######################################################################################################################################################
for i in range(len(numbers)):
        for j in range(i + 1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
   

In [11]:
!cat /kaggle/working/qwen_samples_10.jsonl


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{"task_id": "HumanEval/0", "completion": "for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False"}
{"task_id": "HumanEval/0", "completion": "for_close = set()\n    for_far = []\n    \n    for_close.update(numbers)\n    \n    for_far.append(sorted(to_close))\n    \n    return len(set(range(len(for_far)))) < 2"}
{"task_id": "HumanEval/0", "completion": "for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False"}
{"task_id": "HumanEval/0", "completion": "for_next in range(len(numbers)):\n        current = numbers[current]\n        next_num = numbers[next]\n\n        # Calculate the absolute difference between the current number and the next number\n        diff = abs(current - next)\n\n        # If the difference is less than or equal to the th